# Load and Enrichment to Azure Cognitive Search

In this Jupyter Notebook, we create and run enrichment steps to unlock searchable content in the specified Azure blob. It performs operations over mixed content in Azure Storage, such as images and application files, using a skillset that analyzes and extracts text information that becomes searchable in Azure Cognitive Search. 
The reference sample can be found at [Tutorial: Use Python and AI to generate searchable content from Azure blobs](https://docs.microsoft.com/azure/search/cognitive-search-tutorial-blob-python).

Although only  PDF files are used here, this can be done at a much larger scale and Azure Cognitive Search supports a range of other file formats including: Microsoft Office (DOCX/DOC, XSLX/XLS, PPTX/PPT, MSG), HTML, XML, ZIP, and plain text files (including JSON).

This notebook creates the following objects on your search service:

+ search index
+ data source
+ skillset
+ indexer

In the last step, you'll run queries against the search index to explore the text output that was generated for each blob.

This notebook calls the [Search REST APIs](https://docs.microsoft.com/rest/api/searchservice/), but you can also use the Azure.Search.Documents client library in the Azure SDK for Python to perform the same steps. See this [Python quickstart](https://docs.microsoft.com/azure/search/search-get-started-python) for details.

To run this notebook, you should have already uploade the sample data to a blob container in Azure Storage account. Replace the placeholders for the search service endpoint, the admin API key, Azure Storage connection string, and blob container. Once you've provided all four values, you can run all cells, but the query won't return results until the indexer is finished and the search index is loaded. 

We recommend running each step and making sure it completes before moving on.

Reference:

https://learn.microsoft.com/en-us/azure/search/cognitive-search-tutorial-blob

https://github.com/Azure-Samples/azure-search-python-samples/blob/main/Tutorial-AI-Enrichment/PythonTutorial-AzureSearch-AIEnrichment.ipynb

In [1]:
import os
import json
import requests
from pprint import pprint

In [11]:
# Get the Accounts and Keys

api_version = '2021-04-30-Preview'

# endpoint = os.environ.get("SEARCH_ENDPOINT")
# api_key = os.getenv("SEARCH_KEY")
# datasourceConnectionString = os.getenv("DATASOURCE_CONNECTION_STRING")
# cog_services_name = os.getenv("COGNITIVE_SERVICES_ACCOUNT_NAME")
# cog_services_key = os.getenv("COGNITIVE_SERVICES_ACCOUNT_KEY")

endpoint = "https://azure-cog-search-pabdyosydd7ta.search.windows.net"
api_key = "DDDUwtXOCSOjm1fPBRLNFofKEEvxXDpGF0Sy4S3ktjAzSeAgbz9Q"
datasourceConnectionString = "DefaultEndpointsProtocol=https;AccountName=arxivdatasetcs;AccountKey=M0f/46RGw1IvIpSEXuR7hyprzwvVBiCvIbKYNIlbtJzD2X96KBegKZp59pg4soiu2hSjtRXfhl/5+AStsWkLPA==;EndpointSuffix=core.windows.net"
cog_services_name = "cognitive-service-pabdyosydd7ta"
cog_services_key = "54c7c95ed3a74e2da7694dbd995a093c"

In [13]:
# Define the names for the data source, skillset, index and indexer
datasource_name = "cogsrch-datasource"
skillset_name = "cogsrch-skillset"
index_name = "cogsrch-index"
indexer_name = "cogsrch-indexer"

In [14]:
# Setup the Payloads header
api_version = '2021-04-30-Preview'
headers = {'Content-Type': 'application/json','api-key': api_key}
params = {'api-version': api_version}

## Create Data Source (Blob container with the Arxiv CS pdfs)

In [15]:
# Create a data source
# This data source points to your Azure Storage account.
# You should already have a blob container that contains the sample data

datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities.",
    "type": "azureblob",
    "credentials": {
        "connectionString": datasourceConnectionString
    },
    "container": {
        "name": "pdf"
    }
}
r = requests.put(endpoint + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


## Create Skillset - OCR, Text Splitter, Language Detection, KeyPhrase extraction, Entity Recognition

https://learn.microsoft.com/en-us/azure/search/cognitive-search-working-with-skillsets

https://learn.microsoft.com/en-us/azure/search/cognitive-search-predefined-skills

In [16]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description": "Extract entities, detect language and extract key-phrases",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
            "description": "Extract text (plain and structured) from image.",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "detectOrientation": True,
            "inputs": [
                {
                  "name": "image",
                  "source": "/document/normalized_images/*"
                }
            ],
                "outputs": [
                {
                  "name": "text"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
            "description": "Create merged_text, which includes all the textual representation of each image inserted at the right location in the content field. This is useful for PDF and other file formats that supported embedded images.",
            "context": "/document",
            "insertPreTag": " ",
            "insertPostTag": " ",
            "inputs": [
                {
                  "name":"text", "source": "/document/content"
                },
                {
                  "name": "itemsToInsert", "source": "/document/normalized_images/*/text"
                },
                {
                  "name":"offsets", "source": "/document/normalized_images/*/contentOffset"
                }
            ],
            "outputs": [
                {
                  "name": "mergedText", 
                  "targetName" : "merged_text"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",
            "maximumPageLength": 4000,
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/merged_text"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "pages"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
            "description": "If you have multilingual content, adding a language code is useful for filtering",
            "context": "/document",
            "inputs": [
                {
                  "name": "text",
                  "source": "/document/merged_text"
                }
            ],
            "outputs": [
                {
                  "name": "languageName",
                  "targetName": "language"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
            "context": "/document/pages/*",
            "maxKeyPhraseCount": 2,
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/pages/*"
                }
            ],
            "outputs": [
                {
                    "name": "keyPhrases",
                    "targetName": "keyPhrases"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.V3.EntityRecognitionSkill",
            "context": "/document",
            "categories": ["Person", "Location", "Organization", "DateTime", "URL", "Email"],
            "minimumPrecision": 0.3, 
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/merged_text"
                }
            ],
            "outputs": [
                {
                    "name": "persons", 
                    "targetName": "persons"
                },
                {
                    "name": "locations", 
                    "targetName": "locations"
                },
                {
                    "name": "organizations", 
                    "targetName": "organizations"
                },
                {
                    "name": "dateTimes", 
                    "targetName": "dateTimes"
                },
                {
                    "name": "urls", 
                    "targetName": "urls"
                },
                {
                    "name": "emails", 
                    "targetName": "emails"
                }
            ]
        }
    ],
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": cog_services_name,
        "key": cog_services_key
    }
}

r = requests.put(endpoint + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


## Create Index

The body of the request defines the schema of the search index. A fields collection requires one field to be designated as the key. For blob content, this field is often the "metadata_storage_path" that uniquely identifies each blob in the container.

In this schema, the "text" field receives OCR output, "content" receives merged output, "language" receives language detection output. Key phrases, entities, and several fields lifted from blob storage comprise the remaining entries.

In [17]:
# Create an index
# Queries operate over the searchable fields and filterable fields in the index
index_payload = {
    "name": index_name,
    "fields": [
        {
            "name": "content",
            "type": "Edm.String",
            "searchable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "text",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "sortable": "false",
            "filterable": "true",
            "facetable": "false"
        },
        {
            "name": "language",
            "type": "Edm.String",
            "searchable": "true",
            "sortable": "true",
            "filterable": "true",
            "facetable": "false"
        },
        {
            "name": "keyPhrases",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "sortable": "false",
            "filterable": "true",
            "facetable": "true"
        },
        {
            "name": "persons",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "sortable": "false",
            "filterable": "true",
            "facetable": "true"
        },
        {
            "name": "locations",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "sortable": "false",
            "filterable": "true",
            "facetable": "true"
        },
        {
            "name": "organizations",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "sortable": "false",
            "filterable": "true",
            "facetable": "true"
        },
        {
            "name": "dateTimes",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "sortable": "false",
            "filterable": "true",
            "facetable": "true"
        },
        {
            "name": "urls",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "emails",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "sortable": "false",
            "filterable": "true",
            "facetable": "true"
        },
        {
            "name": "metadata_storage_path",
            "type": "Edm.String",
            "key": "true",
            "searchable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "metadata_storage_name",
            "type": "Edm.String",
            "searchable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false"
            }
    ],
    "semantic": {
      "configurations": [
        {
          "name": "my-semantic-config",
          "prioritizedFields": {
            "prioritizedContentFields": [
                {
                    "fieldName": "content"
                }
                ]
          }
        }
      ]
    }
}

r = requests.put(endpoint + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


## Create and Run the Indexer - (runs the pipeline)
This process takes about 30 mins to load all the Arxiv CS pds

Call Create Indexer to drive the pipeline. The three components you have created thus far (data source, skillset, index) are inputs to an indexer. Creating the indexer on Azure Cognitive Search is the event that puts the entire pipeline into motion.

In [18]:
# Create an indexer
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "fieldMappings": [
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "metadata_storage_path",
          "mappingFunction" : { "name" : "base64Encode" }
        },
        {
            "sourceFieldName": "metadata_storage_name",
            "targetFieldName": "metadata_storage_name"
        }
    ],
    "outputFieldMappings":
    [
        {
            "sourceFieldName": "/document/merged_text",
            "targetFieldName": "content"
        },
        {
            "sourceFieldName" : "/document/normalized_images/*/text",
            "targetFieldName" : "text"
        },
        {
            "sourceFieldName": "/document/language",
            "targetFieldName": "language"
        },
        {
            "sourceFieldName": "/document/pages/*/keyPhrases/*",
            "targetFieldName": "keyPhrases"
        },
        {
          "sourceFieldName" : "/document/persons", 
          "targetFieldName" : "persons"
        },
        {
          "sourceFieldName" : "/document/locations", 
          "targetFieldName" : "locations"
        },
        {
            "sourceFieldName": "/document/organizations",
            "targetFieldName": "organizations"
        },
        {
            "sourceFieldName": "/document/dateTimes",
            "targetFieldName": "dateTimes"
        },
        {
            "sourceFieldName": "/document/urls",
            "targetFieldName": "urls"
        },
        {
            "sourceFieldName": "/document/emails",
            "targetFieldName": "emails"
        }
    ],
    "parameters":
    {
        "maxFailedItems": -1,
        "maxFailedItemsPerBatch": -1,
        "configuration":
        {
            "dataToExtract": "contentAndMetadata",
            "imageAction": "generateNormalizedImages"
        }
    }
}

r = requests.put(endpoint + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


In [19]:
# Optionally, get indexer status to confirm that it's running
r = requests.get(endpoint + "/indexers/" + indexer_name +
                 "/status", headers=headers, params=params)
# pprint(json.dumps(r.json(), indent=1))
print(r.status_code)
print("Status:",r.json().get('lastResult').get('status'))
print("Items Processed:",r.json().get('lastResult').get('itemsProcessed'))
print(r.ok)


200
Status: success
Items Processed: 9887
True


## Sample queries

In [20]:
# Query the service for the index definition
# Query responses can be verbose. If you get "Output exceeds the size limit. Open the full output data in a text editor", open the output in an editor.
# r = requests.get(endpoint + "/indexes/" + index_name,
#                  headers=headers, params=params)
# pprint(json.dumps(r.json(), indent=1))

In [21]:
# Query the index to return the contents of "organizations", created through Entity Recognition during enrichment
# For keyword search, replace the asterisk with comma-separated query terms: search=microsoft,azure
# r = requests.get(endpoint + "/indexes/" + index_name +
#                  "/docs?&search=*&$select=organizations", headers=headers, params=params)
# pprint(json.dumps(r.json(), indent=1))

In [46]:
import urllib
from IPython.display import display, HTML

url = endpoint + '/indexes/'+ index_name + '/docs'
url += '?api-version={}'.format(api_version)
url += '&search=what is artficial inteligence'
url += '&select=content'
url += '&$top=5'
url += '&queryLanguage=en-us'
url += '&queryType=semantic'
url += '&semanticConfiguration=my-semantic-config'
url += '&$count=true'
url += '&speller=lexicon'
url += '&answers=extractive|count-3'
url += '&captions=extractive|highlight-true'
url += '&highlightPreTag=' + urllib.parse.quote('<span style="background-color: #f5e8a3">', safe='')
url += '&highlightPostTag=' + urllib.parse.quote('</span>', safe='')

resp = requests.get(url, headers=headers)
print(url)
print(resp.status_code)

search_results = resp.json()
print("Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))
print("Highest Search Score: {}".format(search_results['value'][0]['@search.score']))

https://azure-cog-search-pabdyosydd7ta.search.windows.net/indexes/cogsrch-index/docs?api-version=2021-04-30-Preview&search=what is artficial inteligence&select=content&$top=5&queryLanguage=en-us&queryType=semantic&semanticConfiguration=my-semantic-config&$count=true&speller=lexicon&answers=extractive|count-3&captions=extractive|highlight-true&highlightPreTag=%3Cspan%20style%3D%22background-color%3A%20%23f5e8a3%22%3E&highlightPostTag=%3C%2Fspan%3E
200
Results Found: 9795, Results Returned: 5
Highest Search Score: 24.213554


In [47]:
# i= 0
# for r in search_results.items():
#     pprint(r)
#     i+=1
#     if i==4: break

In [48]:
# Answers from semantic Search
display(HTML('<h3>Top Answers</h3>'))
for result in search_results['@search.answers']:
    if result['score'] > 0.5:
        display(HTML('<h5>' + 'Answer - score: ' + str(result['score']) + '</h5>'))
        display(HTML(result['text']))

# Results from semantic search

contents = dict()

print("\n\n")
display(HTML('<h3>Top Results</h3>'))
for result in search_results['value']:
    if result['@search.rerankerScore'] > 1:
        display(HTML('<h5>' + result['metadata_storage_name'] + '&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;score: ' + str(result['@search.rerankerScore']) + '</h5>'))
        for caption in result['@search.captions']:
            # print(caption)
            display(HTML(caption['text']))
        contents[result['metadata_storage_name']]=result['content']

# Answers with Azure OpenAI

Of course we want OpenAI to give a better answer chat style, so we instead of sending this results, we send the content of this articles to OpenAI and lets GPT model give the answer.

The problem is that the content of the search result files is or can be very lengthy, more than the 4096 tokens allowed by the GPT Azure OpenAI models. So what we need to do is to split in chunks, vectorize and do a vector semantic search. 
This is what the APP will do

In [49]:
contents["0212030v1.pdf"]

"\nError and Voting Submitted to the Journal of Experimental and Theoretical Artificial Intelligence\n\nMay 13, 1993 1\n\nTheoretical Analyses of Cross-Validation Error and\n\nVoting in Instance-Based Learning\n\nPeter Turney\nKnowledge Systems Laboratory\n\nInstitute for Information Technology\nNational Research Council Canada\n\nOttawa, Ontario, Canada\nK1A 0R6\n\n613-993-8564\npeter@ai.iit.nrc.ca\n\nRunning Head: Error and Voting\n\n\n\nError and Voting Submitted to the Journal of Experimental and Theoretical Artificial Intelligence\n\nMay 13, 1993 2\n\nTheoretical Analyses of Cross-Validation Error and\n\nVoting in Instance-Based Learning\n\nAbstract\n\nThis paper begins with a general theory of error in cross-validation testing of algorithms\n\nfor supervised learning from examples. It is assumed that the examples are described by\n\nattribute-value pairs, where the values are symbolic. Cross-validation requires a set of\n\ntraining examples and a set of testing examples. The valu

# References

https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents/samples

https://learn.microsoft.com/en-us/azure/search/search-get-started-python

https://github.com/Azure-Samples/azure-search-python-samples/blob/main/Tutorial-AI-Enrichment/PythonTutorial-AzureSearch-AIEnrichment.ipynb

https://benalexkeen.com/searching-document-text-at-scale-using-azure-cognitive-search/
